In [ ]:
'''Awaiting approval for LLAMA usage'''
! pip install -U git+https://github.com/huggingface/accelerate.git
from vllm import LLM, SamplingParams
import sqlite3
from huggingface_hub import login
import transformers
import torch

login(token = '')

con = sqlite3.connect("/Users/shreyanakum/Documents/NSF@Oulu/Code-Cloning-Analysis/dev/data/embeddings.db")
con.text_factory = lambda b: b.decode(errors = 'ignore') # and here

cur = con.cursor()

  Cloning https://github.com/huggingface/accelerate.git to /private/var/folders/y8/tjhxjlzj4rq0l60s_sy7fl2m0000gn/T/pip-req-build-e56qvke_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /private/var/folders/y8/tjhxjlzj4rq0l60s_sy7fl2m0000gn/T/pip-req-build-e56qvke_
  Resolved https://github.com/huggingface/accelerate.git to commit 0af621bbecc0e43f5d43766a4945d3d2236bb8a9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.8.0.dev0-py3-none-any.whl size=364208 sha256=1e1e45a80ccf45b6bba2d93f781001e61e7fde822e8755ed991e3295fbcaf409
  Stored in directory: /private/var/folders/y8/tjhxjlzj4rq0l60s_sy7fl2m0000gn/T/pip-ephem-wheel-cache-_pqnxu9a/wheels/5a/20/fb/1221fe933b56fe7ac69fd00159d9a1950bc8ced38198abc18f
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installatio

/Users/shreyanakum/Documents/NSF@Oulu/Code-Cloning-Analysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 06-16 11:59:15 [importing.py:17] Triton not installed or not compatible; certain GPU-related functions will not be available.
WARNING 06-16 11:59:15 [importing.py:29] Triton is not installed. Using dummy decorators. Install it via `pip install triton` to enable kernel compilation.
INFO 06-16 11:59:17 [__init__.py:244] Automatically detected platform cpu.


2025-06-16 11:59:18,988	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
res = cur.execute("SELECT id, data FROM PriA")
code_contents = res.fetchall()

In [4]:
! vllm serve meta-llama/Llama-3.1-8B-Instruct

WARNING 06-16 12:01:21 [env_override.py:17] NCCL_CUMEM_ENABLE is set to 0, skipping override. This may increase memory overhead with cudagraph+allreduce: https://github.com/NVIDIA/nccl/issues/1234
INFO 06-16 12:01:21 [importing.py:17] Triton not installed or not compatible; certain GPU-related functions will not be available.
WARNING 06-16 12:01:21 [importing.py:29] Triton is not installed. Using dummy decorators. Install it via `pip install triton` to enable kernel compilation.
INFO 06-16 12:01:22 [__init__.py:244] Automatically detected platform cpu.
INFO 06-16 12:01:25 [config.py:1980] Disabled the custom all-reduce kernel because it is not supported on current platform.
INFO 06-16 12:01:26 [config.py:1980] Disabled the custom all-reduce kernel because it is not supported on current platform.
INFO 06-16 12:01:26 [config.py:1980] Disabled the custom all-reduce kernel because it is not supported on current platform.
INFO 06-16 12:01:26 [config.py:1980] Disabled the custom all-reduce k

In [5]:
prompts = [
    f"How similar are these two code snippets {code_contents[0]} and {code_contents[1]}?",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

llm = LLM(model="meta-llama/Llama-3.1-8B-Instruct")

outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

con.close()

INFO 06-16 15:31:17 [config.py:823] This model supports multiple tasks: {'classify', 'reward', 'embed', 'generate', 'score'}. Defaulting to 'generate'.
WARNING 06-16 15:31:17 [config.py:3220] Your device 'cpu' doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 06-16 15:31:17 [config.py:3271] Casting torch.bfloat16 to torch.float16.
INFO 06-16 15:31:17 [arg_utils.py:1653] cpu is experimental on VLLM_USE_V1=1. Falling back to V0 Engine.
WARNING 06-16 15:31:17 [arg_utils.py:1490] The model has a long context length (131072). This may causeOOM during the initial memory profiling phase, or result in low performance due to small KV cache size. Consider setting --max-model-len to a smaller value.
INFO 06-16 15:31:17 [config.py:1980] Disabled the custom all-reduce kernel because it is not supported on current platform.
WARNING 06-16 15:31:18 [cpu.py:135] Environment variable VLLM_CPU_KVCACHE_SPACE (GiB) for CPU backend is not set, using 4 by default.
INFO 

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.63it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:07<00:08,  4.45s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:17<00:06,  6.77s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:29<00:00,  8.77s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:29<00:00,  7.28s/it]


INFO 06-16 15:31:50 [default_loader.py:272] Loading weights took 29.16 seconds
INFO 06-16 15:31:50 [executor_base.py:113] # cpu blocks: 2048, # CPU blocks: 0


INFO 06-16 15:31:50 [executor_base.py:118] Maximum concurrency for 131072 tokens per request: 0.25x


ValueError: The model's max seq len (131072) is larger than the maximum number of tokens that can be stored in KV cache (32768). Try increasing `VLLM_CPU_KVCACHE_SPACE` or decreasing `max_model_len` when initializing the engine.